In [ ]:
# Repository setup for portable, repo-relative paths
from pathlib import Path
import sys

def _find_repo_root(start: Path | None = None) -> Path:
    start = (start or Path.cwd()).resolve()
    for candidate in [start, *start.parents]:
        if (candidate / "pyproject.toml").exists():
            return candidate
    return start

REPO_ROOT = _find_repo_root()
if str(REPO_ROOT / "src") not in sys.path:
    sys.path.insert(0, str(REPO_ROOT / "src"))

from dx_chat_entropy.paths import get_paths
PATHS = get_paths(REPO_ROOT)


ModerBERT

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
%pip install -U sentence-transformers
%pip install srsly
#%pip install triton - dont' need with mac m2
%pip install git+https://github.com/huggingface/transformers.git # install new huggingface transformers
# [ ] At some point - the need for the newest transformers won't be required once modernBERT is integrated into the main release

%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/metal.html

import torch

if torch.backends.mps.is_available():
    device = torch.device("mps")
    print(f"Using device: {device}")
else:
    print("MPS backend not available. Falling back to CPU.")


In [ ]:
from sentence_transformers import SentenceTransformer

# Download from the 🤗 Hub
model = SentenceTransformer("joe32140/ModernBERT-base-msmarco")
# Note - for my medical project, seems like maybe using a different model would be better
# also may need to retinker the sentence transformer part

# Run inference
sentences = [
    'what county is hayden in',
    "Hayden is a city in Kootenai County, Idaho, United States. Located in the northern portion of the state, just north of Coeur d'Alene, its population was 13,294 at the 2010 census.",
    "According to the United States Census Bureau, the city has a total area of 9.61 square miles (24.89 km2), of which 9.60 square miles (24.86 km2) is land and 0.01 square miles (0.03 km2) is water. It lies at the southwestern end of Hayden Lake, and the elevation of the city is 2,287 feet (697 m) above sea level. Hayden is located on U.S. Route 95 at the junction of Route 41. It is also four miles (6 km) north of Interstate 90 and Coeur d'Alene. The Coeur d'Alene airport is northwest of Hayden.",
]

embeddings = model.encode(sentences)
print(embeddings.shape)
# [3, 768]

# Get the similarity scores for the embeddings
similarities = model.similarity(embeddings, embeddings)
print(similarities.shape)

In [ ]:
#!pip install transformers[sentencepiece]
from transformers import pipeline

text = "Angela Merkel is a politician in Germany and leader of the CDU"
hypothesis_template = "This text is about {}" # Based on this interview, does the patient have {} sign or symptom
classes_verbalized = ["politics", "economy", "entertainment", "environment"]

zeroshot_classifier = pipeline("zero-shot-classification", model="MoritzLaurer/deberta-v3-large-zeroshot-v2.0")

In [ ]:
?pipeline

In [ ]:
%%time
zeroshot_classifier(text, classes_verbalized, hypothesis_template=hypothesis_template, multi_label=False)

In [ ]:
!wget https://raw.githubusercontent.com/koaning/arxiv-frontpage/refs/heads/main/data/annot/new-dataset.jsonl

In [ ]:
import srsly
inputs = list(srsly.read_jsonl("new-dataset.jsonl"))
texts = [d["text"] for d in inputs]
labels = [d["cats"]["new-dataset"] for d in inputs]

In [ ]:
%%time

outputs = zeroshot_classifier(texts, ["new dataset"], hypothesis_template=hypothesis_template, multi_label=True)

In [ ]:
import numpy as np
pred_zs = np.array([o["scores"] for o in outputs]).reshape(-1) > 0.5
np.mean(pred_zs == labels)

In [ ]:
zeroshot_classifier = pipeline("zero-shot-classification", model="MoritzLaurer/ModernBERT-large-zeroshot-v2.0")

In [ ]:
%%time

zeroshot_classifier(text, classes_verbalized, hypothesis_template=hypothesis_template, multi_label=False)

In [ ]:
%%time
outputs = zeroshot_classifier(texts, ["new dataset"], hypothesis_template=hypothesis_template, multi_label=True)

In [ ]:
pred_zs = np.array([o["scores"] for o in outputs]).reshape(-1) > 0.5
np.mean(pred_zs == labels)

In [ ]:
zeroshot_classifier = pipeline("zero-shot-classification", model="MoritzLaurer/ModernBERT-base-zeroshot-v2.0")

In [ ]:
%%time

outputs = zeroshot_classifier(texts, ["new dataset"], hypothesis_template=hypothesis_template, multi_label=True)

In [ ]:
pred_zs = np.array([o["scores"] for o in outputs]).reshape(-1) > 0.5
np.mean(pred_zs == labels)

In [ ]:
SEED_GLOBAL = 42
import os
import random

import torch
if torch.backends.mps.is_available():
    device = torch.device("mps")
    print(f"Using device: {device}")
else:
    print("MPS backend not available. Falling back to CPU.")

from sentence_transformers import SentenceTransformer
from transformers import pipeline
import srsly


np.random.seed(SEED_GLOBAL)
torch.manual_seed(SEED_GLOBAL)
random.seed(SEED_GLOBAL)

import llm

#Anothe
"""
from datasets import load_dataset, concatenate_datasets
import re
import time

import tqdm
import torch
from torch.utils.data import DataLoader

import transformers
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers import TrainingArguments, Trainer
from datasets import ClassLabel
from datasets import load_dataset, load_metric, Dataset, DatasetDict, concatenate_datasets, list_metrics

from sklearn.metrics import balanced_accuracy_score, precision_recall_fscore_support, accuracy_score, classification_report

import gc
from accelerate.utils import release_memory

import wandb
import json
from datetime import datetime
import argparse

from mdutils import MdUtils

# suppress unnecessary tokenizer warning https://github.com/huggingface/transformers/issues/14285
transformers.logging.set_verbosity_error()

"""

In [ ]:
# Load API key from environment only (do not read shell config or print secrets)
openai_api_key = os.getenv("OPENAI_API_KEY")
if not openai_api_key:
    raise ValueError("OPENAI_API_KEY is not set in environment.")


In [ ]:
# verify torch backend
if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    x = torch.ones(1, device=mps_device)
    print (x)
else:
    print ("MPS device not found.")

In [ ]:
from pypdf import PdfReader

# Path to your PDF file
pdf_path = str(PATHS.raw / 'chatbot_transcripts' / 'Intermtn MS4 1 Transcript.pdf')

# Initialize PDF reader
reader = PdfReader(pdf_path)

# Extract text from each page
all_text = ""
for page in reader.pages:
    all_text += page.extract_text()

# Output the text
print(all_text)

Comparison to using OpenAI

In [ ]:
from openai import OpenAI
from markitdown import MarkItDown

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
md = MarkItDown(llm_client=client, llm_model="gpt-4o-mini")
supported_extensions = ('.pptx', '.docx', '.pdf', '.jpg', '.jpeg', '.png')
files_to_convert = [f for f in os.listdir('.') if f.lower().endswith(supported_extensions)]
for file in files_to_convert:
    print(f"\nConverting {file}...")
    try:
        md_file = os.path.splitext(file)[0] + '.md'
        result = md.convert(file)
        with open(md_file, 'w') as f:
            f.write(result.text_content)
        
        print(f"Successfully converted {file} to {md_file}")
    except Exception as e:
        print(f"Error converting {file}: {str(e)}")

print("\nAll conversions completed!")

In [ ]:
hypothesis_template = "This text is about {}" # Based on this interview, does the patient have {} sign or symptom
inputs = list(srsly.read_jsonl("new-dataset.jsonl"))
texts = [d["text"] for d in inputs]
labels = [d["cats"]["new-dataset"] for d in inputs]

In [ ]:
# Print the first 1000 entries of texts and labels
for i, (text, label) in enumerate(zip(texts[:1000], labels[:1000]), start=1):
    print(f"Entry {i}:")
    print(f"Text: {text}")
    print(f"Label: {label}")
    print("-" * 50)  # Separator for better readability

In [ ]:
zeroshot_classifier = pipeline("zero-shot-classification", model="MoritzLaurer/ModernBERT-large-zeroshot-v2.0")

In [ ]:
%%time
outputs = zeroshot_classifier(texts, ["new dataset"], hypothesis_template=hypothesis_template, multi_label=True)

In [ ]:
pred_zs = np.array([o["scores"] for o in outputs]).reshape(-1) > 0.5
np.mean(pred_zs == labels)

Mostly AI synthetic data engine

In [ ]:
from pathlib import Path
import pandas as pd
from mostlyai import engine

# set up workspace
ws = PATHS.archive / "legacy_external" / "ws-tabular-flat"

# load original data
url = "https://github.com/mostly-ai/public-demo-data/raw/refs/heads/dev/census"
trn_df = pd.read_csv(f"{url}/census.csv.gz")

# execute the engine steps
engine.split(                         # split data as PQT files for `trn` + `val` to `{ws}/OriginalData/tgt-data`
  workspace_dir=ws,
  tgt_data=trn_df,
  model_type="TABULAR",
)
engine.analyze(workspace_dir=ws)      # generate column-level statistics to `{ws}/ModelData/tgt-stats/stats.json`
engine.encode(workspace_dir=ws)       # encode training data to `{ws}/OriginalData/encoded-data`
engine.train(                         # train model and store to `{ws}/ModelStore/model-data`
    workspace_dir=ws,
    max_training_time=1,              # limit TRAIN to 1 minute for demo purposes
)
engine.generate(workspace_dir=ws)     # use model to generate synthetic samples to `{ws}/SyntheticData`
pd.read_parquet(ws / "SyntheticData") # load synthetic data

In [ ]:
import pandas as pd
from mostlyai.sdk import MostlyAI

# 1) Initialize the SDK in local or client mode
mostly = MostlyAI(local=True)
# mostly = MostlyAI(base_url='https://app.mostly.ai', api_key='YOUR_API_KEY')

# 2) Load your original data
trn_df = pd.read_csv('https://github.com/mostly-ai/public-demo-data/raw/dev/census/census.csv.gz')

# 3) Train a synthetic data generator
g = mostly.train(name='census', data=trn_df)  # shorthand syntax for 1-table config

# 4) Live probe small synthetic samples
df_samples = mostly.probe(g, size=10)

# 5) Generate a full synthetic dataset
sd = mostly.generate(g, size=100_000)
syn_df = sd.data()

In [ ]:

# Plan - use markdown; explore querying OpenAI, then explore modernBERT


# [ ] TODO:  figure out why this 

from transformers import AutoTokenizer, AutoModelForMaskedLM

model_id = "answerdotai/ModernBERT-base"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForMaskedLM.from_pretrained(model_id)

text = "The capital of France is [MASK]."
inputs = tokenizer(text, return_tensors="pt")
outputs = model(**inputs)

# To get predictions for the mask:
masked_index = inputs["input_ids"][0].tolist().index(tokenizer.mask_token_id)
predicted_token_id = outputs.logits[0, masked_index].argmax(axis=-1)
predicted_token = tokenizer.decode(predicted_token_id)
print("Predicted token:", predicted_token)
# Predicted token:  Paris


Anthropic Calls - 

[ ] todo: still need to troubleshoot the API key and buy credits if going to use

In [ ]:
import anthropic
import os 

# Access the API key
anthropic_api_key = os.getenv("ANTHROPIC_API_KEY")

if not anthropic_api_key:
    raise ValueError("ANTHROPIC_API_KEY environment variable not set!")

client = anthropic.Anthropic(api_key=anthropic_api_key)

response = client.messages.create(
    model="claude-3-5-sonnet-20241022",
    max_tokens=1024,
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "document",
                    "source": {
                        "type": "text",
                        "media_type": "text/plain",
                        "data": "The grass is green. The sky is blue."
                    },
                    "title": "My Document",
                    "context": "This is a trustworthy document.",
                    "citations": {"enabled": True}
                },
                {
                    "type": "text",
                    "text": "What color is the grass and sky?"
                }
            ]
        }
    ]
)
print(response)

### Code from Allen Downey giving an introduction to various regressions and displays

In [ ]:
from os.path import basename, exists

"""Supporting code for Elements of Data Science

by Allen Downey

MIT License
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import contextlib
import gzip
import io
import re
import textwrap
import os

from IPython.core.magic import register_cell_magic
from IPython.core.magic_arguments import argument, magic_arguments, parse_argstring

from os.path import basename, exists
from statsmodels.nonparametric.smoothers_lowess import lowess


# Make the figures smaller to save some screen real estate.
# The figures generated for the book have DPI 400, so scaling
# them by a factor of 4 restores them to the size in the notebooks.
plt.rcParams["figure.dpi"] = 75
plt.rcParams["figure.figsize"] = [6, 3.5]


def wrap(obj):
    for line in textwrap.wrap(str(obj), subsequent_indent="    "):
        print(line)


class FixedWidthVariables(object):
    """Represents a set of variables in a fixed width file."""

    def __init__(self, variables, index_base=0):
        """Initializes.

        variables: DataFrame
        index_base: are the indices 0 or 1 based?

        Attributes:
        colspecs: list of (start, end) index tuples
        names: list of string variable names
        """
        self.variables = variables
        self.colspecs = variables[["start", "end"]] - index_base

        # convert colspecs to a list of pair of int
        self.colspecs = self.colspecs.astype(np.int).values.tolist()
        self.names = variables["name"]

    def read_fixed_width(self, filename, **options):
        """Reads a fixed width ASCII file.

        filename: string filename

        returns: DataFrame
        """
        df = pd.read_fwf(filename, colspecs=self.colspecs, names=self.names, **options)
        return df


def read_stata_dict(dct_file, **options):
    """Reads a Stata dictionary file.

    dct_file: string filename
    options: dict of options passed to open()

    returns: FixedWidthVariables object
    """
    type_map = dict(
        byte=int, int=int, long=int, float=float, double=float, numeric=float
    )

    var_info = []
    with open(dct_file, **options) as f:
        for line in f:
            match = re.search(r"_column\(([^)]*)\)", line)
            if not match:
                continue
            start = int(match.group(1))
            t = line.split()
            vtype, name, fstring = t[1:4]
            name = name.lower()
            if vtype.startswith("str"):
                vtype = str
            else:
                vtype = type_map[vtype]
            long_desc = " ".join(t[4:]).strip('"')
            var_info.append((start, vtype, name, fstring, long_desc))

    columns = ["start", "type", "name", "fstring", "desc"]
    variables = pd.DataFrame(var_info, columns=columns)

    # fill in the end column by shifting the start column
    variables["end"] = variables.start.shift(-1)
    variables.loc[len(variables) - 1, "end"] = 0

    dct = FixedWidthVariables(variables, index_base=1)
    return dct


def read_stata(dct_name, dat_name, **options):
    """Reads Stata files from the given directory.

    dirname: string

    returns: DataFrame
    """
    dct = read_stata_dict(dct_name)
    df = dct.read_fixed_width(dat_name, **options)
    return df


def sample_rows(df, nrows, replace=False):
    """Choose a sample of rows from a DataFrame.

    df: DataFrame
    nrows: number of rows
    replace: whether to sample with replacement

    returns: DataDf
    """
    indices = np.random.choice(df.index, nrows, replace=replace)
    sample = df.loc[indices]
    return sample


def resample_rows(df):
    """Resamples rows from a DataFrame.

    df: DataFrame

    returns: DataFrame
    """
    return sample_rows(df, len(df), replace=True)


def resample_rows_weighted(df, column="finalwgt"):
    """Resamples a DataFrame using probabilities proportional to given column.

    df: DataFrame
    column: string column name to use as weights

    returns: DataFrame
    """
    weights = df[column].copy()
    weights /= sum(weights)
    indices = np.random.choice(df.index, len(df), replace=True, p=weights)
    sample = df.loc[indices]
    return sample


def resample_by_year(df, column="wtssall"):
    """Resample rows within each year.

    df: DataFrame
    column: string name of weight variable

    returns DataFrame
    """
    grouped = df.groupby("year")
    samples = [resample_rows_weighted(group, column) for _, group in grouped]
    sample = pd.concat(samples, ignore_index=True)
    return sample


def values(series):
    """Count the values and sort.

    series: pd.Series

    returns: series mapping from values to frequencies
    """
    return series.value_counts(dropna=False).sort_index()


def count_by_year(gss, varname):
    """Groups by category and year and counts.

    gss: DataFrame
    varname: string variable to group by

    returns: DataFrame with one row per year, one column per category.
    """
    grouped = gss.groupby([varname, "year"])
    count = grouped[varname].count().unstack(level=0)

    # note: the following is not ideal, because it does not
    # distinguish 0 from NA, but in this dataset the only
    # zeros are during years when the question was not asked.
    count = count.replace(0, np.nan).dropna()
    return count


def fill_missing(df, varname, badvals=[98, 99]):
    """Fill missing data with random values.

    df: DataFrame
    varname: string column name
    badvals: list of values to be replaced
    """
    # replace badvals with NaN
    df[varname].replace(badvals, np.nan, inplace=True)

    # get the index of rows missing varname
    null = df[varname].isnull()
    n_missing = sum(null)

    # choose a random sample from the non-missing values
    fill = np.random.choice(df[varname].dropna(), n_missing, replace=True)

    # replace missing data with the samples
    df.loc[null, varname] = fill

    # return the number of missing values replaced
    return n_missing


def round_into_bins(df, var, bin_width, high=None, low=0):
    """Rounds values down to the bin they belong in.

    df: DataFrame
    var: string variable name
    bin_width: number, width of the bins

    returns: array of bin values
    """
    if high is None:
        high = df[var].max()

    bins = np.arange(low, high + bin_width, bin_width)
    indices = np.digitize(df[var], bins)
    return bins[indices - 1]


def underride(d, **options):
    """Add key-value pairs to d only if key is not in d.

    d: dictionary
    options: keyword args to add to d
    """
    for key, val in options.items():
        d.setdefault(key, val)

    return d


def decorate(**options):
    """Decorate the current axes.
    Call decorate with keyword arguments like
    decorate(title='Title',
             xlabel='x',
             ylabel='y')
    The keyword arguments can be any of the axis properties
    https://matplotlib.org/api/axes_api.html
    In addition, you can use `legend=False` to suppress the legend.
    And you can use `loc` to indicate the location of the legend
    (the default value is 'best')
    """
    loc = options.pop("loc", "best")
    if options.pop("legend", True):
        legend(loc=loc)

    plt.gca().set(**options)
    plt.tight_layout()


def legend(**options):
    """Draws a legend only if there is at least one labeled item.
    options are passed to plt.legend()
    https://matplotlib.org/api/_as_gen/matplotlib.pyplot.legend.html
    """
    underride(options, loc="best")

    ax = plt.gca()
    handles, labels = ax.get_legend_handles_labels()
    if handles:
        ax.legend(handles, labels, **options)


def make_lowess(series, frac=2 / 3):
    """Use LOWESS to compute a smooth line.

    series: pd.Series

    returns: pd.Series
    """
    y = series.values
    x = series.index.values

    smooth = lowess(y, x, frac=frac)
    index, data = np.transpose(smooth)
    return pd.Series(data, index=index)


def plot_lowess(series, color, frac=0.7, **options):
    """Plot a smooth line.

    series: pd.Series
    color: string or tuple
    """
    if "label" not in options:
        options["label"] = series.name

    smooth = make_lowess(series, frac=frac)
    smooth.plot(color=color, **options)


def plot_series_lowess(series, color, frac=0.7, **options):
    """Plots a series of data points and a smooth line.

    series: pd.Series
    color: string or tuple
    """
    if "label" not in options:
        options["label"] = series.name

    x = series.index
    y = series.values

    if len(series) == 1:
        # just plot the point
        plt.plot(x, y, "o", color=color, alpha=0.5, label=options["label"])
    else:
        # plot the points and line
        plt.plot(x, y, "o", color=color, alpha=0.5, label="_")
        plot_lowess(series, color, frac, **options)


def plot_columns_lowess(df, columns, colors):
    """Plot the columns in a DataFrame.

    df: pd.DataFrame
    columns: list of column names, in the desired order
    colors: mapping from column names to colors
    """
    for col in columns:
        series = df[col]
        plot_series_lowess(series, colors[col])


def anchor_legend(x, y):
    """Put the legend at the given locationself.

    x: axis coordinate
    y: axis coordinate
    """
    plt.legend(bbox_to_anchor=(x, y), loc="upper left", ncol=1)


def download(url):
    filename = basename(url)
    if not exists(filename):
        from urllib.request import urlretrieve

        local, _ = urlretrieve(url, filename)
        print("Downloaded " + local)


def read_gss(dict_file="GSS.dct", data_file="GSS.dat.gz"):
    from statadict import parse_stata_dict

    download(
        "https://github.com/AllenDowney/"
        + "ElementsOfDataScience/raw/master/data/"
        + dict_file
    )

    download(
        "https://github.com/AllenDowney/"
        + "ElementsOfDataScience/raw/master/data/"
        + data_file
    )

    stata_dict = parse_stata_dict(dict_file)
    fp = gzip.open(data_file)
    gss = pd.read_fwf(fp, names=stata_dict.names, colspecs=stata_dict.colspecs)
    return gss


def traceback(mode):
    """Set the traceback mode.

    mode: string
    """
    # this context suppresses the output
    with contextlib.redirect_stdout(io.StringIO()):
        get_ipython().run_cell(f"%xmode {mode}")


traceback("Minimal")


def extract_function_name(text):
    """Find a function definition and return its name.

    text: String

    returns: String or None
    """
    pattern = r"def\s+(\w+)\s*\("
    match = re.search(pattern, text)
    if match:
        func_name = match.group(1)
        return func_name
    else:
        return None


@register_cell_magic
def expect_error(line, cell):
    try:
        get_ipython().run_cell(cell)
    except Exception as e:
        get_ipython().run_cell("%tb")


@magic_arguments()
@argument("exception", help="Type of exception to catch")
@register_cell_magic
def expect(line, cell):
    args = parse_argstring(expect, line)
    exception = eval(args.exception)
    try:
        get_ipython().run_cell(cell)
    except exception as e:
        get_ipython().run_cell("%tb")



def download(url):
    filename = basename(url)
    if not exists(filename):
        from urllib.request import urlretrieve

        local, _ = urlretrieve(url, filename)
        print("Downloaded " + str(local))
    return filename

download('https://raw.githubusercontent.com/AllenDowney/ElementsOfDataScience/v1/utils.py')

In [ ]:
import pandas as pd
from skimpy import skim

download('https://github.com/AllenDowney/ElementsOfDataScience/' +
         'raw/v1/data/gss_extract_2022.hdf');
gss = pd.read_hdf('gss_extract_2022.hdf', 'gss')
skim(gss)
gss.head()

In [ ]:
# Simple Regression w scipy
from scipy.stats import linregress

data = gss.dropna(subset=['realinc', 'educ'])
xs = data['educ']
ys = data['realinc']

res = linregress(xs, ys)
res._asdict()

In [ ]:
# simple regression with statsmodels
import statsmodels.formula.api as smf

results = smf.ols('realinc ~ educ', data=data).fit()
type(results)
results.params

In [ ]:
# multiple regress statsmodel
results = smf.ols('realinc ~ educ + age', data=gss).fit()
results.params

In [ ]:
# group by age
import matplotlib.pyplot as plt

grouped = gss.groupby('age')
type(grouped)
mean_income_by_age = grouped['realinc'].mean()

plt.plot(mean_income_by_age, 'o', alpha=0.5)
plt.xlabel('Age (years)')
plt.ylabel('Income (1986 $)')
plt.title('Average income, grouped by age');

In [ ]:
# Quadratic age and ed
import numpy as np

gss['age2'] = gss['age']**2
gss['educ2'] = gss['educ']**2
model = smf.ols('realinc ~ educ + educ2 + age + age2', data=gss)
results = model.fit()
results.params

df = pd.DataFrame()
df['age'] = np.linspace(18, 89)
df['age2'] = df['age']**2
df['educ'] = 12
df['educ2'] = df['educ']**2

pred12 = results.predict(df)

plt.plot(mean_income_by_age, 'o', alpha=0.5)
plt.plot(df['age'], pred12, label='High school', color='C4')

df['educ'] = 16
df['educ2'] = df['educ']**2
pred16 = results.predict(df)

df['educ'] = 14
df['educ2'] = df['educ']**2
pred14 = results.predict(df)

plt.plot(mean_income_by_age, 'o', alpha=0.5)
plt.plot(df['age'], pred16, ':', label='Bachelor')
plt.plot(df['age'], pred14, '--', label='Associate')
plt.plot(df['age'], pred12, label='High school', color='C4')

plt.xlabel('Age (years)')
plt.ylabel('Income (1986 $)')
plt.title('Income versus age, grouped by education level')
plt.legend();

In [ ]:
#categorical - sex

formula = 'realinc ~ educ + educ2 + age + age2 + C(sex)'
results = smf.ols(formula, data=gss).fit()
results.params

In [ ]:
#
gss['gunlaw'] = gss['gunlaw'].replace([2], [0]) # make binary 0 = no, 1= yes
gss['gunlaw'].value_counts()
formula = 'gunlaw ~ age + age2 + educ + educ2 + C(sex)'
results = smf.logit(formula, data=gss).fit()
results.params

df = pd.DataFrame()
df['age'] = np.linspace(18, 89)
df['educ'] = 12
df['age2'] = df['age']**2
df['educ2'] = df['educ']**2

df['sex'] = 1
pred_male = results.predict(df)
df['sex'] = 2
pred_female = results.predict(df)

grouped = gss.groupby('age')
favor_by_age = grouped['gunlaw'].mean()

plt.plot(favor_by_age, 'o', alpha=0.5)
plt.plot(df['age'], pred_female, label='Female')
plt.plot(df['age'], pred_male, '--', label='Male')

plt.xlabel('Age')
plt.ylabel('Probability of favoring gun law')
plt.title('Support for gun law versus age, grouped by sex')
plt.legend();

3b1b solver for Wordle - which relies on entropy calculations.



In [ ]:
!git clone https://github.com/woctezuma/3b1b-wordle-solver.git
%cd 3b1b-wordle-solver
%pip install -q -r requirements.txt

In [ ]:
url="https://github.com/woctezuma/3b1b-wordle-solver/releases/download/wordle/pattern_matrix.npy"
!wget {url} -P data/wordle

In [ ]:
url="https://github.com/woctezuma/3b1b-wordle-solver/releases/download/dungleon/pattern_matrix.npy"
!wget {url} -P data/dungleon

In [ ]:
!python simulations.py -h

In [ ]:
!python simulations.py --game-name wordle

In [ ]:
!python simulations.py --game-name dungleon --first-guess ZBCFS --test-answer WSGCO --hard-mode

In [ ]:
%cp data/dungleon/possible_words.seen.txt data/dungleon/possible_words.txt

In [ ]:
!python simulations.py --game-name dungleon --first-guess AWOCS --test-answer VMIFI --hard-mode

In [ ]:
%cp data/dungleon/possible_words.seen.txt data/dungleon/possible_words.txt
%cp data/dungleon/possible_words.seen.txt data/dungleon/allowed_words.txt

In [ ]:
FOLDER_NAME = "simulations"
%mkdir -p {FOLDER_NAME}

In [ ]:
from pathlib import Path

for l in Path('data/dungleon/possible_words.seen.txt').read_text().split('\n'):
  if l:
    !python simulations.py --game-name dungleon --hard-mode --first-guess {l} | tail > {FOLDER_NAME}/{l}.txt

In [ ]:
keyword = "Total"

!grep {keyword} {FOLDER_NAME}/*.txt

### Med SSS Clinical reasoning - THUS FAR, NOT SUCCESFUL IN GETTING THIS GOING

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
from collections import OrderedDict

In [ ]:
from huggingface_hub import list_repo_files

# List all files in the repository
files = list_repo_files("pixas/MedSSS_Policy")
print("Files in repository:")
print(files)

In [ ]:
from huggingface_hub import hf_hub_download
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel, PeftConfig
from safetensors.torch import load_file

# Load base model
base_model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-3.1-8B-Instruct",
    torch_dtype="auto",
    device_map="auto"
)

# Download the adapter weights
adapter_path = hf_hub_download(repo_id="pixas/MedSSS_Policy", filename="adapter_model.safetensors")
adapter_state_dict = load_file(adapter_path)

# Adjust keys in the adapter state_dict
from collections import OrderedDict

adjusted_state_dict = OrderedDict()
for key, value in adapter_state_dict.items():
    new_key = key.replace("model.model.model", "model")
    adjusted_state_dict[new_key] = value

# Load the adjusted adapter state_dict into the base_model
base_model.load_state_dict(adjusted_state_dict, strict=False)

peft_config = PeftConfig(
    base_model_name_or_path="meta-llama/Llama-3.1-8B-Instruct",
    task_type="CAUSAL_LM",  # Task type (e.g., causal language modeling)
    peft_type="LORA",       # Type of PEFT (LoRA in this case)
    inference_mode=True     # Set to True for inference
)

# Wrap the base_model with PeftModel
model = PeftModel(base_model, peft_config=peft_config)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.1-8B-Instruct")

# Test the model
input_text = "How to stop a cough?"
inputs = tokenizer(input_text, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=50)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [ ]:
base_model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-3.1-8B-Instruct",  # Correct base model
    torch_dtype="auto",
    device_map="auto"
)

for name, param in base_model.named_parameters():
    print(name)

# Load the LoRA adapter state_dict
lora_state_dict = torch.load("path/to/lora/checkpoint.bin", map_location="cpu")

# Create a new state_dict with adjusted keys
new_state_dict = OrderedDict()
for key, value in lora_state_dict["model"].items():
    # Adjust keys by replacing the extra 'model.model.model' with just 'model'
    new_key = key.replace("model.model.model", "model")
    new_state_dict[new_key] = value

# Load the adjusted state_dict into the model
model.load_state_dict(new_state_dict, strict=False)



model = PeftModel.from_pretrained(
    base_model, 
    "pixas/MedSSS_Policy", 
    torch_dtype="auto", 
    ignore_mismatched_sizes=True
)



model.embed_tokens.weight
base_model.model.embed_tokens.weight

#tokenizer = AutoTokenizer.from_pretrained("pixas/MedSSS_Policy")
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.1-8B-Instruct")

input_text = "How to stop a cough?"
messages = [{"role": "user", "content": input_text}]
inputs = tokenizer(tokenizer.apply_chat_template(messages, tokenize=False,add_generation_prompt=True
), return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=2048)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [ ]:
# To check model specification for troubleshooting

from huggingface_hub import hf_hub_download, model_info

model_card = hf_hub_download(repo_id="pixas/MedSSS_Policy", filename="README.md")
with open(model_card, "r") as f:
    print(f.read())

# Fetch model information
info = model_info("pixas/MedSSS_Policy")

# Print information
print(f"Model ID: {info.modelId}")
print(f"Tags: {info.tags}")
print(f"Files: {info.siblings}")
print(f"Card Data: {info.cardData}")

In [ ]:
from peft import PeftConfig

# Load adapter config
adapter_config = PeftConfig.from_pretrained("pixas/MedSSS_Policy")
print(adapter_config)

In [ ]:
%%time
# Ollama Version - requires ollama to be installed (Mac only) and ~>16gb ram, 8gb of disk space.
# Local - too verbose and doesn't quite get the instructions right (openAI has better prompt engineering)

model = llm.get_model("hf.co/unsloth/DeepSeek-R1-Distill-Llama-8B-GGUF:Q8_0") 

# Create a new column to store the responses
transcripts_df["responses"] = None

# Iterate through each transcript in the DataFrame
for i, row in transcripts_df.iterrows():
    full_prompts = row["full_prompts"]  # Get the full_prompts dictionary for this transcript
    transcript_responses = {}  # Dictionary to store responses for this transcript
    
    # Iterate through each diagnosis and its associated prompt
    for diagnosis, prompt in full_prompts.items():
        response = model.prompt(prompt)  # Get the model's response
        transcript_responses[diagnosis] = response.text()  # Store the response text
    
    # Save the responses back into the DataFrame
    transcripts_df.at[i, "responses"] = transcript_responses

display(transcripts_df)
#print(tabulate(transcripts_df, headers = 'keys', tablefmt = 'fancy_grid'))

### Bambi and PyMC

https://bambinos.github.io/bambi/

In [ ]:
import arviz as az
import bambi as bmb
import numpy as np
import pandas as pd
import skimpy

In [ ]:
# Read in a dataset from the package content
data = bmb.load_data("sleepstudy")

# See first rows
data.head()
skimpy.skim(data)
 
# Initialize the fixed effects only model
model = bmb.Model('Reaction ~ Days', data)

# Get model description
print(model)

# Fit the model using 1000 on each chain
results = model.fit(draws=1000)

# Key summary and diagnostic info on the model parameters
az.summary(results)

# Use ArviZ to plot the results
az.plot_trace(results)

In [ ]:
data = pd.DataFrame({
    "g": np.random.choice(["Yes", "No"], size=50),
    "x1": np.random.normal(size=50),
    "x2": np.random.normal(size=50)
})
skimpy.skim(data)